# Synthetic anomaly video generation

## Purpose

This software is designed to generate synthetic video movement data (blocks on the screen) at a specified anomaly rate.

It will also model state changes (when completed). For example, an office may have the following states:

1) Normal business day
2) After hours - no-one around
3) After hours - cleaners

Each state will have a different anomaly rate and different behaviours

It outputs individual frames and then combines these into an .mp4 file.

## Prerequisites

1) Pygame - from Pygame.org
2) libav - can be installed using Homebrew: brew install libav
3) Homebrew can be installed at command line with: /usr/bin/ruby -e "$(curl -fsSL https://raw.githubusercontent.com/Homebrew/install/master/install)"


## Changes needed

1) Define a main section, put other code in procedures
2) Make program independent of predefined paths
3) Set maximum number of frames to generate
4) Allow definition of multiple states

In [ ]:
# Initialise pygame and load needed libraries
import pygame, random, sys, os, glob, datetime
from random import randint
from random import random
from random import choice
from pygame.locals import *
from time import time

start = time()

def addsprite():
    "Instantiates a new sprite and applies anomalies to it, or adds sprites. Can't change an existing sprite"
    if random() < anomaly_probability:
        # Select from the anomaly array
        i = randint(0,9)
        n=len(sprites)
        # Fill next slot, if all slots are filled, replace the oldest sprite
        if n >= 99:
            n=1   
        # add to selectedsprites at first available location
        selectedsprites[n] = (anomalysprites[i][0],anomalysprites[i][1],anomalysprites[i][2],anomalysprites[i][3],(anomalysprites[i][4][0], anomalysprites[i][4][1]),anomalysprites[i][5])
        mysprite = Block(selectedsprites[n][0],selectedsprites[n][1],selectedsprites[n][2],selectedsprites[n][3],(selectedsprites[n][4][0], selectedsprites[n][4][1]),selectedsprites[n][5])
        sprites.add(mysprite)
        print("Anomaly at frame:",frames)
    else:
        # Select from the standard array
        i = randint(0,9)
        # add to selectedsprites at first available location - n
        n=len(sprites)
        # Fill next slot, if all slots are filled, replace the oldest sprite
        if n >= 99:
            n=1   
        selectedsprites[n] = (normalsprites[i][0],normalsprites[i][1],normalsprites[i][2],normalsprites[i][3],(normalsprites[i][4][0], normalsprites[i][4][1]),normalsprites[i][5])
        mysprite = Block(selectedsprites[n][0],selectedsprites[n][1],selectedsprites[n][2],selectedsprites[n][3],(selectedsprites[n][4][0], selectedsprites[n][4][1]), selectedsprites[n][5])
        sprites.add(mysprite)        
    
class Block(pygame.sprite.Sprite):
    def __init__(self, sprite_x_size, sprite_y_size, sprite_x_move, sprite_y_move, pos,sprite_colour):
        "Sets up a sprite based on parameters, colour is white"
        pygame.sprite.Sprite.__init__(self)
        # set sprite size: x,y
        self.image = pygame.surface.Surface((sprite_x_size, sprite_y_size))
        if sprite_colour == "white":
            self.image.fill((255, 255, 255))
        elif sprite_colour == "blue":
            self.image.fill((0,191,255))
        #set position
        self.rect = self.image.get_rect(center=pos)
        self.dir = 4

    def update(self):
        # Off the screen?
        if not screen.get_rect().contains(self.rect):
            self.kill()
            addsprite()
            
    def moveXY(self,moveX,moveY):
        # Moves a sprite
        self.rect.x += moveX
        self.rect.y += moveY
        # Off the screen?
        if not screen.get_rect().contains(self.rect):
            self.kill()
            addsprite()

# ----------------------------------------------------------------------------------------------            
# Specify global variables / settings
anomaly_probability = 0.00025 #probability of an anomaly event being triggered, default is 0.000025
frames = 1
numsprites = 9 #max is currently 10
normalsprites = [None]*10
anomalysprites = [None]*10
selectedsprites = [None]*100 # make big enough to handle new sprites
pygame.DOUBLEBUF
pygame.HWSURFACE

# Clean up old files
fileList=glob.glob('/Users/graemewoods/Documents/mSynthetic data generation/Outputs/*.png')
for filePath in fileList:
    try:
        os.remove(filePath)
    except:
        print("Error while deleting file : ",filePath)
        
# Instantiate normal sprites - size x, size y, move x, move y, pos x, pos y
normalsprites[0] = (20,30,1,0,(10,100),"white")
normalsprites[1] = (10,20,-1,0,(590,200),"white")
normalsprites[2] = (30,20,-2,0,(590,250),"white")
normalsprites[3] = (40,30,1,0,(10,300),"white")
normalsprites[4] = (10,20,3,0,(10,350),"white")
normalsprites[5] = (15,15,1,0,(10,320),"white")
normalsprites[6] = (20,10,2,0,(10,250),"white")
normalsprites[7] = (20,10,3,0,(10,35),"white")
normalsprites[8] = (10,10,1,0,(10,45),"white")
normalsprites[9] = (20,20,1,0,(10,125),"white")

# Instantiate anomaly sprites - size x, size y, move x, move y, pos x, pos y
anomalysprites[0] = (70,5,-1,-15,(590,470),"blue")
anomalysprites[1] = (10,20,1,0,(520,400),"blue")
anomalysprites[2] = (20,20,12,0,(570,450),"blue")
anomalysprites[3] = (30,30,1,-30,(10,400),"blue")
anomalysprites[4] = (50,20,10,2,(50,425),"blue")
anomalysprites[5] = (20,10,3,3,(400,415),"blue")
anomalysprites[6] = (20,40,12,10,(590,470),"blue")
anomalysprites[7] = (20,50,-5,1,(10,400),"blue")
anomalysprites[8] = (20,60,20,0,(590,400),"blue")
anomalysprites[9] = (20,20,1,40,(590,400),"blue")

# Specify activities

pygame.init()
screen = pygame.display.set_mode((640, 480))
pygame.display.set_caption('iCetana synthetic anomaly video generation: Running')
run = True

sprites = pygame.sprite.Group()
# Initialise first group sprites
for i in range(0,numsprites):
    selectedsprites[i] = (normalsprites[i][0],normalsprites[i][1],normalsprites[i][2],normalsprites[i][3],(normalsprites[i][4][0], normalsprites[i][4][1]),normalsprites[i][5])
    mysprite = Block(normalsprites[i][0],normalsprites[i][1],normalsprites[i][2],normalsprites[i][3],(normalsprites[i][4][0], normalsprites[i][4][1]),normalsprites[i][5])
    sprites.add(mysprite)

# ----------------------------------------------------------------------------------------------                
# Run until user stops by pressing Return
while run:
    for e in pygame.event.get():
        if e.type == pygame.KEYDOWN: #check if end
            run = False
            end = time()
            # Measure frames per second
            print ("FPS: %f" % (frames / ((end - start))))
            # Convert frames to .mp4 video
            pygame.display.set_caption('iCetana synthetic anomaly video generation: Converting')
            os.system("avconv -r 10 -f image2 -i Outputs/%04d.png -y -qscale 0 -s 640x480 -aspect 4:3 Outputs/anomalyvideo.mp4")
            # rename base output video file
            try:
                os.rename("Outputs/anomalyvideo.mp4",'Outputs/anomalyvideo' + datetime.datetime.now().strftime("%Y%m%d-%H%M") + '.mp4')
            except:
                print("Existing anomalyvideo not present")
            pygame.display.set_caption('iCetana synthetic anomaly video generation: Stopped')
            pygame.quit()
                
    # move
    i=0
    # print("length of sprites is: ",len(sprites))
    for each_sprite in sprites:
        moveX = selectedsprites[i][2]
        moveY = selectedsprites[i][3]
        each_sprite.moveXY(moveX,moveY)
        i += 1
    
    # update
    screen.fill((0, 0, 0))
    sprites.draw(screen)
    sprites.update()
    frames +=1
    # redraw screen
    #pygame.display.flip()
    pygame.display.update()

    # Output frames data to directory
    outputfile = "Outputs/%04d.png" % frames
    pygame.image.save(screen, outputfile)